In [1]:
!nvidia-smi

Mon May  9 10:42:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 27.2 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.0 MB/s 
     |████████████████████████████████| 880 kB 50.2 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8660ce046fdeb55af87d1e319f0680fc2b69995207f9b315100347f83ff39211
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 25.6 MB/s 
     |████████████████████████████████| 136 kB 65.9 MB/s 
     |████████████████████████████████| 1.1 MB 61.0 MB/s 
     |████████████████████████████████| 212 kB 75.8 MB/s 
     |████████████████████████████

In [3]:
from google.colab import drive
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets import ClassLabel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import pandas as pd
from datasets import load_metric
from transformers import Trainer
from sklearn.metrics import classification_report

drive.mount('/content/drive')
google_path = 'drive/MyDrive/'

Mounted at /content/drive


In [4]:
dataset = load_dataset('csv', 
                       data_files={'train': google_path+"train.csv",'test': google_path+"val.csv"},
                       delimiter='\t'
                       )
dataset

Using custom data configuration default-4c182b5ded536034


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4c182b5ded536034/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 89996
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [5]:
all_langs = [
    'ru',
    'uk',
    'ka',
    'he',
    'en',
    'de',
    'be',
    'kk',
    'az',
    'hy',
]

labels = ClassLabel(num_classes=10, names=all_langs)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(batch):
    tokens = tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)
    tokens['label'] = labels.str2int(batch['label'])
    return tokens

tokenized_datasets = dataset.map(tokenize, batched=True)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

  0%|          | 0/90 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [6]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
     num_labels=10, 
     problem_type="single_label_classification"
)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [8]:
batch_size = 16

training_args = TrainingArguments(
    output_dir=google_path+"lang-xlm-roberta-base",
    overwrite_output_dir=True,
    logging_strategy = "epoch",
    evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    save_steps=5000,
    save_total_limit=2,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    report = classification_report(predictions, labels, target_names=all_langs, output_dict=True)
    acc = report['accuracy']
    report.pop('accuracy', None)
    scores = {f"{lang}_{metric}": score for lang, lang_report in report.items() for metric, score in lang_report.items()}
    return {'accuracy': acc, **scores}

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 89996
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11250


Epoch,Training Loss,Validation Loss,Accuracy,Ru Precision,Ru Recall,Ru F1-score,Ru Support,Uk Precision,Uk Recall,Uk F1-score,Uk Support,Ka Precision,Ka Recall,Ka F1-score,Ka Support,He Precision,He Recall,He F1-score,He Support,En Precision,En Recall,En F1-score,En Support,De Precision,De Recall,De F1-score,De Support,Be Precision,Be Recall,Be F1-score,Be Support,Kk Precision,Kk Recall,Kk F1-score,Kk Support,Az Precision,Az Recall,Az F1-score,Az Support,Hy Precision,Hy Recall,Hy F1-score,Hy Support,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support
1,0.085500,0.040184,0.989300,0.994000,0.971652,0.982699,1023,0.974000,0.992864,0.983342,981,0.999000,1.000000,0.999500,999,1.000000,0.998004,0.999001,1002,0.973000,0.969124,0.971058,1004,0.968000,0.973843,0.970913,994,0.995000,0.997994,0.996495,997,0.995000,0.998996,0.996994,996,0.997000,1.000000,0.998498,997,0.998000,0.991063,0.994519,1007,0.989300,0.989354,0.989302,10000,0.989347,0.989300,0.989298,10000
2,0.027900,0.031517,0.990700,0.985000,0.987964,0.986480,997,0.988000,0.984064,0.986028,1004,0.999000,0.999000,0.999000,1000,1.000000,0.999001,0.999500,1001,0.981000,0.970326,0.975634,1011,0.970000,0.981781,0.975855,988,0.999000,0.986180,0.992548,1013,0.996000,1.000000,0.997996,996,0.998000,0.998999,0.998499,999,0.991000,1.000000,0.995480,991,0.990700,0.990732,0.990702,10000,0.990723,0.990700,0.990698,10000


Saving model checkpoint to drive/MyDrive/lang-xlm-roberta-base/checkpoint-5000
Configuration saved in drive/MyDrive/lang-xlm-roberta-base/checkpoint-5000/config.json
Model weights saved in drive/MyDrive/lang-xlm-roberta-base/checkpoint-5000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16
Saving model checkpoint to drive/MyDrive/lang-xlm-roberta-base/checkpoint-10000
Configuration saved in drive/MyDrive/lang-xlm-roberta-base/checkpoint-10000/config.json
Model weights saved in drive/MyDrive/lang-xlm-roberta-base/checkpoint-10000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and ha

TrainOutput(global_step=11250, training_loss=0.05672913682725694, metrics={'train_runtime': 5043.953, 'train_samples_per_second': 35.685, 'train_steps_per_second': 2.23, 'total_flos': 1.1840321683771392e+16, 'train_loss': 0.05672913682725694, 'epoch': 2.0})

In [12]:
trainer.save_model()

Saving model checkpoint to drive/MyDrive/lang-xlm-roberta-base
Configuration saved in drive/MyDrive/lang-xlm-roberta-base/config.json
Model weights saved in drive/MyDrive/lang-xlm-roberta-base/pytorch_model.bin


In [13]:
model_path = 'drive/MyDrive/lang-xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(model_path)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.1

In [26]:
input = ["тоби пизда, тикай з городу"]
preds = model(**tokenizer(input, return_tensors='pt')).logits.tolist()
preds = labels.int2str(int(np.argmax(preds[0])))
preds

'uk'